In [4]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [5]:
# =============================================================================
# FIX DATA LEAKAGE - CREATE PROPER SPLITS FROM ORIGINAL DATASET
# =============================================================================

def fix_data_leakage_and_create_proper_splits():
    """Fix data leakage by creating proper splits from original dataset"""
    
    print("🚨 FIXING DATA LEAKAGE ISSUE")
    print("=" * 60)
    
    print("PROBLEM IDENTIFIED:")
    print("   ❌ Augmented images from same source appear in train/val/test")
    print("   ❌ This causes data leakage and inflated performance")
    print("   ❌ 90% test accuracy is likely invalid due to this issue")
    
    print("\n🔧 SOLUTION: CREATE PROPER SPLITS FROM ORIGINAL DATA")
    print("=" * 60)
    
    # Step 1: Work with original (non-augmented) dataset
    original_data_path = 'Dataset/'  # Your original dataset
    
    # Collect all original files with their classes
    all_original_files = []
    class_to_id = {}
    
    print("📂 Collecting original files...")
    for class_id, class_name in enumerate(os.listdir(original_data_path)):
        class_path = os.path.join(original_data_path, class_name)
        if os.path.isdir(class_path):
            class_to_id[class_name] = class_id
            
            # Get only original images (not augmented)
            image_files = [f for f in os.listdir(class_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            for img_file in image_files:
                all_original_files.append({
                    'file_name': img_file,
                    'class_name': class_name,
                    'class_id': class_id,
                    'full_path': os.path.join(class_path, img_file)
                })
    
    # Convert to DataFrame
    original_df = pd.DataFrame(all_original_files)
    
    print(f"✅ Collected {len(original_df)} original images")
    print(f"   Classes: {len(class_to_id)}")
    
    # Step 2: Create proper stratified splits (NO DATA LEAKAGE)
    print("\n📊 Creating proper stratified splits...")
    
    # Split at the IMAGE level (not augmented copies)
    train_files, temp_files = train_test_split(
        original_df,
        test_size=0.4,  # 40% for val+test
        stratify=original_df['class_id'],
        random_state=42
    )
    
    val_files, test_files = train_test_split(
        temp_files,
        test_size=0.5,  # Split the 40% equally between val and test
        stratify=temp_files['class_id'],
        random_state=42
    )
    
    print(f"✅ Split sizes:")
    print(f"   Train: {len(train_files)} images ({len(train_files)/len(original_df):.1%})")
    print(f"   Val: {len(val_files)} images ({len(val_files)/len(original_df):.1%})")
    print(f"   Test: {len(test_files)} images ({len(test_files)/len(original_df):.1%})")
    
    # Step 3: Create physical directories with NO LEAKAGE
    clean_split_path = 'Clean_Splits'
    
    # Remove old splits if they exist
    if os.path.exists(clean_split_path):
        shutil.rmtree(clean_split_path)
    
    # Create new directory structure
    for split_name in ['train', 'val', 'test']:
        for class_name in class_to_id.keys():
            os.makedirs(os.path.join(clean_split_path, split_name, class_name), exist_ok=True)
    
    # Copy files to clean splits
    splits_data = {
        'train': train_files,
        'val': val_files,
        'test': test_files
    }
    
    print(f"\n📁 Creating clean split directories...")
    for split_name, split_df in splits_data.items():
        print(f"   Copying {split_name} files...")
        for _, row in tqdm(split_df.iterrows(), total=len(split_df), desc=f"Copying {split_name}"):
            src_path = row['full_path']
            dst_path = os.path.join(clean_split_path, split_name, row['class_name'], row['file_name'])
            shutil.copy2(src_path, dst_path)
    
    # Step 4: Apply augmentation ONLY to training set
    print(f"\n🔄 Applying augmentation ONLY to training set...")
    
    def augment_training_set_only(train_split_path):
        """Apply augmentation only to training set to prevent leakage"""
        
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.15,
            height_shift_range=0.15,
            horizontal_flip=True,
            zoom_range=0.15,
            shear_range=0.1,
            brightness_range=[0.9, 1.1],
            fill_mode='nearest'
        )
        
        # Count original training images per class
        train_counts = {}
        for class_name in os.listdir(train_split_path):
            class_path = os.path.join(train_split_path, class_name)
            if os.path.isdir(class_path):
                count = len([f for f in os.listdir(class_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                train_counts[class_name] = count
        
        # Calculate target size (median of current training sizes)
        target_size = int(np.median(list(train_counts.values())))
        print(f"   Target size per class in training: {target_size}")
        
        # Augment minority classes in training set only
        for class_name, current_count in train_counts.items():
            if current_count < target_size:
                needed = target_size - current_count
                class_path = os.path.join(train_split_path, class_name)
                
                print(f"   Augmenting {class_name}: {current_count} → {target_size} (+{needed})")
                
                # Get original images in this class
                original_images = [f for f in os.listdir(class_path) 
                                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                
                # Generate augmented images
                generated = 0
                max_attempts = needed * 3
                attempts = 0
                
                while generated < needed and attempts < max_attempts:
                    try:
                        # Select random original image
                        base_img = np.random.choice(original_images)
                        img_path = os.path.join(class_path, base_img)
                        
                        # Load and augment
                        img = cv2.imread(img_path)
                        if img is None:
                            attempts += 1
                            continue
                        
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = img.reshape((1,) + img.shape)
                        
                        # Generate augmented image
                        aug_iter = train_datagen.flow(img, batch_size=1)
                        aug_img = next(aug_iter)[0].astype(np.uint8)
                        
                        # Save with clear augmentation prefix
                        base_name = os.path.splitext(base_img)[0]
                        ext = os.path.splitext(base_img)[1]
                        aug_filename = f"TRAIN_AUG_{generated:04d}_{base_name}{ext}"
                        aug_path = os.path.join(class_path, aug_filename)
                        
                        # Save
                        aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(aug_path, aug_img_bgr)
                        
                        generated += 1
                        
                    except Exception as e:
                        attempts += 1
                        continue
                    
                    attempts += 1
    
    # Apply augmentation only to training set
    train_split_path = os.path.join(clean_split_path, 'train')
    augment_training_set_only(train_split_path)
    
    print(f"\n✅ CLEAN SPLITS CREATED - NO DATA LEAKAGE!")
    print(f"   Path: {clean_split_path}")
    print(f"   Augmentation: Applied ONLY to training set")
    print(f"   Val/Test: Original images only - NO augmentation")
    
    return clean_split_path, splits_data, class_to_id

# Execute the fix
clean_split_path, clean_splits_data, clean_class_to_id = fix_data_leakage_and_create_proper_splits()

🚨 FIXING DATA LEAKAGE ISSUE
PROBLEM IDENTIFIED:
   ❌ Augmented images from same source appear in train/val/test
   ❌ This causes data leakage and inflated performance
   ❌ 90% test accuracy is likely invalid due to this issue

🔧 SOLUTION: CREATE PROPER SPLITS FROM ORIGINAL DATA
📂 Collecting original files...
✅ Collected 35725 original images
   Classes: 23

📊 Creating proper stratified splits...
✅ Split sizes:
   Train: 21435 images (60.0%)
   Val: 7145 images (20.0%)
   Test: 7145 images (20.0%)

📁 Creating clean split directories...
   Copying train files...


Copying train: 100%|██████████| 21435/21435 [01:15<00:00, 282.04it/s]


   Copying val files...


Copying val: 100%|██████████| 7145/7145 [00:24<00:00, 293.64it/s]


   Copying test files...


Copying test: 100%|██████████| 7145/7145 [00:24<00:00, 296.75it/s]



🔄 Applying augmentation ONLY to training set...
   Target size per class in training: 1006
   Augmenting Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 985 → 1006 (+21)
   Augmenting Pepper__bell___Bacterial_spot: 598 → 1006 (+408)
   Augmenting Pepper__bell___healthy: 887 → 1006 (+119)
   Augmenting Potato___Early_blight: 600 → 1006 (+406)
   Augmenting Potato___healthy: 91 → 1006 (+915)
   Augmenting Potato___Late_blight: 600 → 1006 (+406)
   Augmenting Tomato_Early_blight: 600 → 1006 (+406)
   Augmenting Tomato_healthy: 955 → 1006 (+51)
   Augmenting Tomato_Leaf_Mold: 571 → 1006 (+435)
   Augmenting Tomato__Target_Spot: 842 → 1006 (+164)
   Augmenting Tomato__Tomato_mosaic_virus: 224 → 1006 (+782)

✅ CLEAN SPLITS CREATED - NO DATA LEAKAGE!
   Path: Clean_Splits
   Augmentation: Applied ONLY to training set
   Val/Test: Original images only - NO augmentation


In [6]:
# =============================================================================
# VALIDATE CLEAN SPLITS - ENSURE NO DATA LEAKAGE
# =============================================================================

def validate_clean_splits(clean_split_path):
    """Validate that clean splits have no data leakage"""
    
    print("\n🔍 VALIDATING CLEAN SPLITS - NO DATA LEAKAGE CHECK")
    print("=" * 60)
    
    # Collect all base filenames from each split
    train_files = set()
    val_files = set()
    test_files = set()
    
    for split, file_set in [('train', train_files), ('val', val_files), ('test', test_files)]:
        split_path = os.path.join(clean_split_path, split)
        
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        # For augmented files, extract original base name
                        if img_file.startswith('TRAIN_AUG_'):
                            # Extract original filename from augmented name
                            base_name = '_'.join(img_file.split('_')[3:])
                        else:
                            # Original file
                            base_name = img_file
                        
                        file_set.add(base_name)
    
    # Check for overlaps
    train_val_overlap = train_files.intersection(val_files)
    train_test_overlap = train_files.intersection(test_files)
    val_test_overlap = val_files.intersection(test_files)
    
    print(f"📊 CLEAN SPLIT VALIDATION RESULTS:")
    print(f"   Train files (unique): {len(train_files)}")
    print(f"   Val files (unique): {len(val_files)}")
    print(f"   Test files (unique): {len(test_files)}")
    print(f"   Train-Val overlap: {len(train_val_overlap)} files")
    print(f"   Train-Test overlap: {len(train_test_overlap)} files")
    print(f"   Val-Test overlap: {len(val_test_overlap)} files")
    
    if len(train_val_overlap) == 0 and len(train_test_overlap) == 0 and len(val_test_overlap) == 0:
        print(f"   ✅ SUCCESS: NO DATA LEAKAGE DETECTED!")
        print(f"   🎉 Clean splits are valid for training")
        return True
    else:
        print(f"   ❌ ERROR: Data leakage still detected")
        return False

# Validate the clean splits
validation_success = validate_clean_splits(clean_split_path)


🔍 VALIDATING CLEAN SPLITS - NO DATA LEAKAGE CHECK
📊 CLEAN SPLIT VALIDATION RESULTS:
   Train files (unique): 21435
   Val files (unique): 7145
   Test files (unique): 7145
   Train-Val overlap: 0 files
   Train-Test overlap: 0 files
   Val-Test overlap: 0 files
   ✅ SUCCESS: NO DATA LEAKAGE DETECTED!
   🎉 Clean splits are valid for training


In [7]:
# =============================================================================
# COMPREHENSIVE PRE-TRAINING VALIDATION CHECKS
# =============================================================================

def comprehensive_pre_training_validation(clean_split_path):
    """Perform comprehensive validation before training"""
    
    print("🔍 COMPREHENSIVE PRE-TRAINING VALIDATION")
    print("=" * 70)
    
    validation_results = {}
    
    # 1. VERIFY IMAGE INTEGRITY
    print("\n1️⃣ CHECKING IMAGE INTEGRITY:")
    print("-" * 40)
    
    def check_image_integrity():
        """Check for corrupted or invalid images"""
        corrupted_files = []
        total_images = 0
        
        for split in ['train', 'val', 'test']:
            split_path = os.path.join(clean_split_path, split)
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    for img_file in os.listdir(class_path):
                        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                            img_path = os.path.join(class_path, img_file)
                            total_images += 1
                            
                            try:
                                # Try to read image
                                img = cv2.imread(img_path)
                                if img is None:
                                    corrupted_files.append(img_path)
                                elif img.shape[0] < 32 or img.shape[1] < 32:  # Too small
                                    corrupted_files.append(img_path)
                            except Exception as e:
                                corrupted_files.append(img_path)
        
        print(f"   Total images checked: {total_images:,}")
        print(f"   Corrupted/invalid images: {len(corrupted_files)}")
        
        if len(corrupted_files) == 0:
            print("   ✅ ALL IMAGES ARE VALID")
            return True
        else:
            print(f"   ⚠️  FOUND {len(corrupted_files)} PROBLEMATIC IMAGES")
            for file_path in corrupted_files[:5]:  # Show first 5
                print(f"      - {file_path}")
            if len(corrupted_files) > 5:
                print(f"      ... and {len(corrupted_files) - 5} more")
            return False
    
    validation_results['image_integrity'] = check_image_integrity()
    
    # 2. CHECK CLASS BALANCE AND DISTRIBUTION
    print("\n2️⃣ CHECKING CLASS BALANCE:")
    print("-" * 40)
    
    def check_class_balance():
        """Check class distribution across splits"""
        split_distributions = {}
        
        for split in ['train', 'val', 'test']:
            split_path = os.path.join(clean_split_path, split)
            class_counts = {}
            
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    count = len([f for f in os.listdir(class_path) 
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    class_counts[class_name] = count
            
            split_distributions[split] = class_counts
        
        # Check if all splits have the same classes
        train_classes = set(split_distributions['train'].keys())
        val_classes = set(split_distributions['val'].keys())
        test_classes = set(split_distributions['test'].keys())
        
        all_classes_consistent = (train_classes == val_classes == test_classes)
        
        # Calculate imbalance ratios
        train_counts = list(split_distributions['train'].values())
        val_counts = list(split_distributions['val'].values())
        test_counts = list(split_distributions['test'].values())
        
        train_imbalance = max(train_counts) / min(train_counts) if min(train_counts) > 0 else float('inf')
        val_imbalance = max(val_counts) / min(val_counts) if min(val_counts) > 0 else float('inf')
        test_imbalance = max(test_counts) / min(test_counts) if min(test_counts) > 0 else float('inf')
        
        print(f"   Classes in train: {len(train_classes)}")
        print(f"   Classes in val: {len(val_classes)}")
        print(f"   Classes in test: {len(test_classes)}")
        print(f"   All splits have same classes: {all_classes_consistent}")
        print(f"   Train imbalance ratio: {train_imbalance:.2f}:1")
        print(f"   Val imbalance ratio: {val_imbalance:.2f}:1")
        print(f"   Test imbalance ratio: {test_imbalance:.2f}:1")
        
        # Check for classes with too few samples
        min_samples_per_class = 5
        problematic_classes = []
        
        for split, counts in split_distributions.items():
            for class_name, count in counts.items():
                if count < min_samples_per_class:
                    problematic_classes.append(f"{split}/{class_name}: {count} samples")
        
        if len(problematic_classes) > 0:
            print(f"   ⚠️  CLASSES WITH TOO FEW SAMPLES:")
            for prob_class in problematic_classes[:10]:  # Show first 10
                print(f"      - {prob_class}")
        
        # Overall assessment
        balance_ok = (all_classes_consistent and 
                     train_imbalance < 10 and 
                     len(problematic_classes) == 0)
        
        if balance_ok:
            print("   ✅ CLASS DISTRIBUTION IS ACCEPTABLE")
        else:
            print("   ⚠️  CLASS DISTRIBUTION ISSUES DETECTED")
        
        return balance_ok, split_distributions
    
    balance_ok, split_distributions = check_class_balance()
    validation_results['class_balance'] = balance_ok
    
    # 3. CHECK AUGMENTATION QUALITY
    print("\n3️⃣ CHECKING AUGMENTATION QUALITY:")
    print("-" * 40)
    
    def check_augmentation_quality():
        """Check if augmented images are reasonable"""
        train_path = os.path.join(clean_split_path, 'train')
        aug_issues = []
        
        # Sample some augmented images to check
        aug_files_checked = 0
        max_check = 100  # Check up to 100 augmented files
        
        for class_name in os.listdir(train_path):
            class_path = os.path.join(train_path, class_name)
            if os.path.isdir(class_path):
                aug_files = [f for f in os.listdir(class_path) 
                           if f.startswith('TRAIN_AUG_')]
                
                for aug_file in aug_files[:10]:  # Check up to 10 per class
                    if aug_files_checked >= max_check:
                        break
                    
                    aug_path = os.path.join(class_path, aug_file)
                    try:
                        img = cv2.imread(aug_path)
                        if img is None:
                            aug_issues.append(f"Cannot read: {aug_path}")
                        elif img.shape[0] < 32 or img.shape[1] < 32:
                            aug_issues.append(f"Too small: {aug_path}")
                        # Add more quality checks if needed
                        aug_files_checked += 1
                    except Exception as e:
                        aug_issues.append(f"Error reading: {aug_path}")
                
                if aug_files_checked >= max_check:
                    break
        
        print(f"   Augmented files checked: {aug_files_checked}")
        print(f"   Issues found: {len(aug_issues)}")
        
        if len(aug_issues) == 0:
            print("   ✅ AUGMENTED IMAGES LOOK GOOD")
            return True
        else:
            print("   ⚠️  AUGMENTATION QUALITY ISSUES:")
            for issue in aug_issues[:5]:
                print(f"      - {issue}")
            return False
    
    validation_results['augmentation_quality'] = check_augmentation_quality()
    
    # 4. CHECK DATASET SIZE ADEQUACY
    print("\n4️⃣ CHECKING DATASET SIZE ADEQUACY:")
    print("-" * 40)
    
    def check_dataset_size():
        """Check if dataset size is adequate for training"""
        total_train = sum(split_distributions['train'].values())
        total_val = sum(split_distributions['val'].values())
        total_test = sum(split_distributions['test'].values())
        num_classes = len(split_distributions['train'])
        
        samples_per_class_train = total_train / num_classes
        samples_per_class_val = total_val / num_classes
        samples_per_class_test = total_test / num_classes
        
        print(f"   Training samples: {total_train:,}")
        print(f"   Validation samples: {total_val:,}")
        print(f"   Test samples: {total_test:,}")
        print(f"   Number of classes: {num_classes}")
        print(f"   Avg samples per class (train): {samples_per_class_train:.1f}")
        print(f"   Avg samples per class (val): {samples_per_class_val:.1f}")
        print(f"   Avg samples per class (test): {samples_per_class_test:.1f}")
        
        # Adequacy thresholds
        min_train_per_class = 50
        min_val_per_class = 10
        min_test_per_class = 10
        
        adequate = (samples_per_class_train >= min_train_per_class and
                   samples_per_class_val >= min_val_per_class and
                   samples_per_class_test >= min_test_per_class)
        
        if adequate:
            print("   ✅ DATASET SIZE IS ADEQUATE")
        else:
            print("   ⚠️  DATASET MIGHT BE TOO SMALL FOR ROBUST TRAINING")
        
        return adequate
    
    validation_results['dataset_size'] = check_dataset_size()
    
    # 5. VERIFY NO DUPLICATE FILES
    print("\n5️⃣ CHECKING FOR DUPLICATE FILES:")
    print("-" * 40)
    
    def check_duplicates():
        """Check for duplicate files within each split"""
        import hashlib
        
        def get_file_hash(file_path):
            """Get MD5 hash of file"""
            hash_md5 = hashlib.md5()
            try:
                with open(file_path, "rb") as f:
                    for chunk in iter(lambda: f.read(4096), b""):
                        hash_md5.update(chunk)
                return hash_md5.hexdigest()
            except:
                return None
        
        duplicates_found = 0
        
        for split in ['train', 'val', 'test']:
            split_path = os.path.join(clean_split_path, split)
            file_hashes = {}
            split_duplicates = 0
            
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    for img_file in os.listdir(class_path):
                        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                            img_path = os.path.join(class_path, img_file)
                            file_hash = get_file_hash(img_path)
                            
                            if file_hash and file_hash in file_hashes:
                                split_duplicates += 1
                                print(f"      Duplicate in {split}: {img_path}")
                            elif file_hash:
                                file_hashes[file_hash] = img_path
            
            print(f"   Duplicates in {split}: {split_duplicates}")
            duplicates_found += split_duplicates
        
        if duplicates_found == 0:
            print("   ✅ NO DUPLICATE FILES FOUND")
            return True
        else:
            print(f"   ⚠️  FOUND {duplicates_found} DUPLICATE FILES")
            return False
    
    validation_results['no_duplicates'] = check_duplicates()
    
    # 6. FINAL VALIDATION SUMMARY
    print("\n🎯 VALIDATION SUMMARY:")
    print("=" * 50)
    
    passed_checks = sum(validation_results.values())
    total_checks = len(validation_results)
    
    print(f"Validation checks passed: {passed_checks}/{total_checks}")
    print()
    
    for check_name, passed in validation_results.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        check_display = check_name.replace('_', ' ').title()
        print(f"   {check_display}: {status}")
    
    # Overall recommendation
    if passed_checks == total_checks:
        print(f"\n🎉 ALL VALIDATIONS PASSED!")
        print("✅ READY FOR TRAINING - Dataset is clean and robust")
        recommendation = "PROCEED_WITH_TRAINING"
    elif passed_checks >= total_checks - 1:
        print(f"\n⚠️  MINOR ISSUES DETECTED")
        print("🟡 TRAINING POSSIBLE - Monitor results carefully")
        recommendation = "PROCEED_WITH_CAUTION"
    else:
        print(f"\n🚨 SIGNIFICANT ISSUES DETECTED")
        print("❌ FIX ISSUES BEFORE TRAINING")
        recommendation = "FIX_ISSUES_FIRST"
    
    return recommendation, validation_results

# Execute comprehensive validation
if validation_success:
    print("✅ Initial data leakage validation passed")
    print("🔍 Running comprehensive pre-training validation...")
    training_recommendation, detailed_validation = comprehensive_pre_training_validation(clean_split_path)
else:
    print("❌ Fix data leakage issues first before proceeding")
    training_recommendation = "FIX_DATA_LEAKAGE_FIRST"

✅ Initial data leakage validation passed
🔍 Running comprehensive pre-training validation...
🔍 COMPREHENSIVE PRE-TRAINING VALIDATION

1️⃣ CHECKING IMAGE INTEGRITY:
----------------------------------------
   Total images checked: 39,838
   Corrupted/invalid images: 0
   ✅ ALL IMAGES ARE VALID

2️⃣ CHECKING CLASS BALANCE:
----------------------------------------
   Classes in train: 23
   Classes in val: 23
   Classes in test: 23
   All splits have same classes: True
   Train imbalance ratio: 1.91:1
   Val imbalance ratio: 21.37:1
   Test imbalance ratio: 20.71:1
   ✅ CLASS DISTRIBUTION IS ACCEPTABLE

3️⃣ CHECKING AUGMENTATION QUALITY:
----------------------------------------
   Augmented files checked: 100
   Issues found: 0
   ✅ AUGMENTED IMAGES LOOK GOOD

4️⃣ CHECKING DATASET SIZE ADEQUACY:
----------------------------------------
   Training samples: 25,548
   Validation samples: 7,145
   Test samples: 7,145
   Number of classes: 23
   Avg samples per class (train): 1110.8
   Avg sam

In [8]:
# =============================================================================
# FINAL TRAINING READINESS ASSESSMENT
# =============================================================================

def final_training_readiness_assessment():
    """Final assessment before training"""
    
    print("\n🎯 FINAL TRAINING READINESS ASSESSMENT")
    print("=" * 60)
    
    if training_recommendation == "PROCEED_WITH_TRAINING":
        print("🟢 STATUS: READY FOR TRAINING")
        print("✅ All validation checks passed")
        print("✅ Dataset is clean and properly structured")
        print("✅ No data leakage detected")
        print("✅ Class distribution is reasonable")
        print("✅ Images are valid and accessible")
        
        print(f"\n🚀 RECOMMENDED NEXT STEPS:")
        print("   1. Proceed with model training")
        print("   2. Monitor training curves carefully")
        print("   3. Use early stopping and learning rate reduction")
        print("   4. Save best model based on validation loss")
        print("   5. Evaluate on test set only after training is complete")
        
        return True
        
    elif training_recommendation == "PROCEED_WITH_CAUTION":
        print("🟡 STATUS: PROCEED WITH CAUTION")
        print("⚠️  Minor issues detected, but training is possible")
        print("✅ Data leakage is fixed")
        print("⚠️  Some validation checks flagged concerns")
        
        print(f"\n⚠️  RECOMMENDATIONS:")
        print("   1. Proceed with training but monitor closely")
        print("   2. Use more conservative hyperparameters")
        print("   3. Implement extra validation during training")
        print("   4. Be prepared to stop and fix issues if needed")
        
        return True
        
    else:
        print("🔴 STATUS: NOT READY FOR TRAINING")
        print("❌ Significant issues must be fixed first")
        
        print(f"\n🛠️  REQUIRED FIXES:")
        if not detailed_validation.get('image_integrity', True):
            print("   - Fix corrupted or invalid images")
        if not detailed_validation.get('class_balance', True):
            print("   - Address class imbalance issues")
        if not detailed_validation.get('augmentation_quality', True):
            print("   - Fix augmentation quality problems")
        if not detailed_validation.get('dataset_size', True):
            print("   - Increase dataset size or reduce number of classes")
        if not detailed_validation.get('no_duplicates', True):
            print("   - Remove duplicate files")
        
        return False

# Final assessment
training_ready = final_training_readiness_assessment()

print(f"\n{'='*60}")
if training_ready:
    print("🎉 VALIDATION COMPLETE - READY TO TRAIN!")
    print("You can now proceed with confidence that your results will be legitimate.")
else:
    print("🛑 VALIDATION INCOMPLETE - PLEASE FIX ISSUES FIRST")
    print("Address the flagged issues before training to ensure valid results.")
print(f"{'='*60}")


🎯 FINAL TRAINING READINESS ASSESSMENT
🟡 STATUS: PROCEED WITH CAUTION
⚠️  Minor issues detected, but training is possible
✅ Data leakage is fixed
⚠️  Some validation checks flagged concerns

⚠️  RECOMMENDATIONS:
   1. Proceed with training but monitor closely
   2. Use more conservative hyperparameters
   3. Implement extra validation during training
   4. Be prepared to stop and fix issues if needed

🎉 VALIDATION COMPLETE - READY TO TRAIN!
You can now proceed with confidence that your results will be legitimate.


In [9]:
# =============================================================================
# CLEAN UP DUPLICATES AND PREPARE FOR TRAINING
# =============================================================================

def clean_up_duplicates_and_finalize(clean_split_path):
    """Clean up any remaining duplicates and prepare for training"""
    
    print("🧹 CLEANING UP DUPLICATES AND FINALIZING DATASET")
    print("=" * 60)
    
    import hashlib
    
    def get_file_hash(file_path):
        """Get MD5 hash of file"""
        hash_md5 = hashlib.md5()
        try:
            with open(file_path, "rb") as f:
                for chunk in iter(lambda: f.read(4096), b""):
                    hash_md5.update(chunk)
            return hash_md5.hexdigest()
        except:
            return None
    
    duplicates_removed = 0
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(clean_split_path, split)
        file_hashes = {}
        
        print(f"🔍 Checking {split} split for duplicates...")
        
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                files_to_remove = []
                
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(class_path, img_file)
                        file_hash = get_file_hash(img_path)
                        
                        if file_hash and file_hash in file_hashes:
                            files_to_remove.append(img_path)
                            duplicates_removed += 1
                        elif file_hash:
                            file_hashes[file_hash] = img_path
                
                # Remove duplicates
                for duplicate_file in files_to_remove:
                    try:
                        os.remove(duplicate_file)
                        print(f"   ✅ Removed duplicate: {os.path.basename(duplicate_file)}")
                    except Exception as e:
                        print(f"   ❌ Error removing {duplicate_file}: {e}")
    
    print(f"\n🎯 CLEANUP RESULTS:")
    print(f"   Duplicates removed: {duplicates_removed}")
    
    if duplicates_removed > 0:
        print("   ✅ Dataset is now clean and ready for training!")
    else:
        print("   ✅ No duplicates found - dataset was already clean!")
    
    return duplicates_removed

# Clean up duplicates
duplicates_removed = clean_up_duplicates_and_finalize(clean_split_path)

🧹 CLEANING UP DUPLICATES AND FINALIZING DATASET
🔍 Checking train split for duplicates...
   ✅ Removed duplicate: 3673d121-b5de-481c-b057-d4ee5b4959b1___RS_HL 6269.JPG
   ✅ Removed duplicate: 9b75de13-d4b0-4b3f-988c-3e9926eef957___RS_HL 6273.JPG
   ✅ Removed duplicate: c21cf428-bfc3-4710-b5d2-69d1c0e94748___RS_HL 6268_flipTB.JPG
   ✅ Removed duplicate: dc18b924-f172-445d-8fed-61445d437aaa___RS_HL 6270.JPG
   ✅ Removed duplicate: 505465db-407b-4e0a-8110-7479dad5261c___GH_HL Leaf 389.JPG
   ✅ Removed duplicate: cfd491d6-4af5-4728-8f0e-0d330a07174a___GH_HL Leaf 482.2.JPG
   ✅ Removed duplicate: e786ac89-29fe-47e3-b49e-b9a9ee7edd9d___GH_HL Leaf 342.1.JPG
   ✅ Removed duplicate: bd4f09bd-ee85-4ab1-bce0-8cde3fdd7f1b___GHLB_PS Leaf 23.7 Day 13.jpg
   ✅ Removed duplicate: c1775bad-7c02-41fb-bb7d-f8df91d60ac3___GHLB_PS Leaf 23.5 Day 13.jpg
   ✅ Removed duplicate: d81682aa-746b-4e07-af2b-52ebb6f4c017___GHLB2 Leaf 102.JPG
   ✅ Removed duplicate: e5d707cd-077c-43af-bda9-6138e516ff51___GHLB2 Leaf 89

In [10]:
# =============================================================================
# RE-VALIDATE AFTER DUPLICATE CLEANUP - FIXED VERSION
# =============================================================================

def re_validate_after_cleanup(clean_split_path):
    """Re-run validation after cleaning up duplicates"""
    
    print("\n🔄 RE-VALIDATING AFTER DUPLICATE CLEANUP")
    print("=" * 60)
    
    # Re-run the comprehensive validation
    validation_results = {}
    
    # 1. Re-check for duplicates
    print("\n1️⃣ RE-CHECKING FOR DUPLICATES:")
    print("-" * 40)
    
    import hashlib
    
    def get_file_hash(file_path):
        """Get MD5 hash of file"""
        hash_md5 = hashlib.md5()
        try:
            with open(file_path, "rb") as f:
                for chunk in iter(lambda: f.read(4096), b""):
                    hash_md5.update(chunk)
            return hash_md5.hexdigest()
        except:
            return None
    
    duplicates_found = 0
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(clean_split_path, split)
        file_hashes = {}
        split_duplicates = 0
        
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(class_path, img_file)
                        file_hash = get_file_hash(img_path)
                        
                        if file_hash and file_hash in file_hashes:
                            split_duplicates += 1
                            print(f"      Duplicate in {split}: {img_path}")
                        elif file_hash:
                            file_hashes[file_hash] = img_path
        
        print(f"   Duplicates in {split}: {split_duplicates}")
        duplicates_found += split_duplicates
    
    if duplicates_found == 0:
        print("   ✅ NO DUPLICATE FILES FOUND - CLEANUP SUCCESSFUL!")
        validation_results['no_duplicates'] = True
    else:
        print(f"   ❌ STILL FOUND {duplicates_found} DUPLICATE FILES")
        validation_results['no_duplicates'] = False
    
    # 2. Re-check data leakage
    print("\n2️⃣ RE-CHECKING DATA LEAKAGE:")
    print("-" * 40)
    
    # Collect all base filenames from each split
    train_files = set()
    val_files = set()
    test_files = set()
    
    for split, file_set in [('train', train_files), ('val', val_files), ('test', test_files)]:
        split_path = os.path.join(clean_split_path, split)
        
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        # For augmented files, extract original base name
                        if img_file.startswith('TRAIN_AUG_'):
                            # Extract original filename from augmented name
                            base_name = '_'.join(img_file.split('_')[3:])
                        else:
                            # Original file
                            base_name = img_file
                        
                        file_set.add(base_name)
    
    # Check for overlaps
    train_val_overlap = train_files.intersection(val_files)
    train_test_overlap = train_files.intersection(test_files)
    val_test_overlap = val_files.intersection(test_files)
    
    print(f"   Train files (unique): {len(train_files)}")
    print(f"   Val files (unique): {len(val_files)}")
    print(f"   Test files (unique): {len(test_files)}")
    print(f"   Train-Val overlap: {len(train_val_overlap)} files")
    print(f"   Train-Test overlap: {len(train_test_overlap)} files")
    print(f"   Val-Test overlap: {len(val_test_overlap)} files")
    
    if len(train_val_overlap) == 0 and len(train_test_overlap) == 0 and len(val_test_overlap) == 0:
        print(f"   ✅ NO DATA LEAKAGE DETECTED!")
        validation_results['no_data_leakage'] = True
    else:
        print(f"   ❌ DATA LEAKAGE STILL DETECTED")
        validation_results['no_data_leakage'] = False
    
    # 3. Get updated dataset statistics (THIS IS THE MISSING PIECE!)
    print("\n3️⃣ UPDATED DATASET STATISTICS:")
    print("-" * 40)
    
    total_images = 0
    split_stats = {}
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(clean_split_path, split)
        split_count = 0
        class_counts = {}
        
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                count = len([f for f in os.listdir(class_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                class_counts[class_name] = count
                split_count += count
        
        split_stats[split] = {
            'total': split_count,
            'classes': class_counts
        }
        total_images += split_count
        
        print(f"   {split.capitalize()}: {split_count:,} images")
    
    print(f"   Total images: {total_images:,}")
    print(f"   Classes: {len(split_stats['train']['classes'])}")
    
    # 4. Check class balance after cleanup (NOW USING THE LOCAL split_stats!)
    print("\n4️⃣ CLASS BALANCE AFTER CLEANUP:")
    print("-" * 40)
    
    train_counts = list(split_stats['train']['classes'].values())
    val_counts = list(split_stats['val']['classes'].values())
    test_counts = list(split_stats['test']['classes'].values())
    
    train_imbalance = max(train_counts) / min(train_counts) if min(train_counts) > 0 else float('inf')
    val_imbalance = max(val_counts) / min(val_counts) if min(val_counts) > 0 else float('inf')
    test_imbalance = max(test_counts) / min(test_counts) if min(test_counts) > 0 else float('inf')
    
    print(f"   Train imbalance ratio: {train_imbalance:.2f}:1")
    print(f"   Val imbalance ratio: {val_imbalance:.2f}:1")
    print(f"   Test imbalance ratio: {test_imbalance:.2f}:1")
    
    # Check for classes with too few samples after cleanup
    min_samples_per_class = 5
    problematic_classes = []
    
    for split, stats in split_stats.items():
        for class_name, count in stats['classes'].items():
            if count < min_samples_per_class:
                problematic_classes.append(f"{split}/{class_name}: {count} samples")
    
    if len(problematic_classes) > 0:
        print(f"   ⚠️  CLASSES WITH TOO FEW SAMPLES:")
        for prob_class in problematic_classes[:10]:
            print(f"      - {prob_class}")
        validation_results['adequate_samples'] = False
    else:
        print(f"   ✅ ALL CLASSES HAVE ADEQUATE SAMPLES")
        validation_results['adequate_samples'] = True
    
    # 5. Check dataset size adequacy after cleanup
    print("\n5️⃣ CHECKING DATASET SIZE ADEQUACY AFTER CLEANUP:")
    print("-" * 40)
    
    total_train = sum(split_stats['train']['classes'].values())
    total_val = sum(split_stats['val']['classes'].values())
    total_test = sum(split_stats['test']['classes'].values())
    num_classes = len(split_stats['train']['classes'])
    
    samples_per_class_train = total_train / num_classes
    samples_per_class_val = total_val / num_classes
    samples_per_class_test = total_test / num_classes
    
    print(f"   Training samples: {total_train:,}")
    print(f"   Validation samples: {total_val:,}")
    print(f"   Test samples: {total_test:,}")
    print(f"   Number of classes: {num_classes}")
    print(f"   Avg samples per class (train): {samples_per_class_train:.1f}")
    print(f"   Avg samples per class (val): {samples_per_class_val:.1f}")
    print(f"   Avg samples per class (test): {samples_per_class_test:.1f}")
    
    # Adequacy thresholds
    min_train_per_class = 50
    min_val_per_class = 10
    min_test_per_class = 10
    
    size_adequate = (samples_per_class_train >= min_train_per_class and
                    samples_per_class_val >= min_val_per_class and
                    samples_per_class_test >= min_test_per_class)
    
    if size_adequate:
        print("   ✅ DATASET SIZE IS ADEQUATE")
        validation_results['dataset_size'] = True
    else:
        print("   ⚠️  DATASET MIGHT BE TOO SMALL FOR ROBUST TRAINING")
        validation_results['dataset_size'] = False
    
    # 6. Final validation summary
    print("\n🎯 POST-CLEANUP VALIDATION SUMMARY:")
    print("=" * 50)
    
    passed_checks = sum(validation_results.values())
    total_checks = len(validation_results)
    
    print(f"Validation checks passed: {passed_checks}/{total_checks}")
    print()
    
    for check_name, passed in validation_results.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        check_display = check_name.replace('_', ' ').title()
        print(f"   {check_display}: {status}")
    
    # Final recommendation
    if passed_checks == total_checks:
        print(f"\n🎉 ALL POST-CLEANUP VALIDATIONS PASSED!")
        print("✅ DATASET IS NOW FULLY CLEAN AND READY FOR TRAINING")
        final_recommendation = "PROCEED_WITH_TRAINING"
    elif passed_checks >= total_checks - 1:
        print(f"\n⚠️  MINOR ISSUES REMAIN")
        print("🟡 TRAINING POSSIBLE BUT MONITOR CAREFULLY")
        final_recommendation = "PROCEED_WITH_CAUTION"
    else:
        print(f"\n🚨 SIGNIFICANT ISSUES REMAIN")
        print("❌ ADDITIONAL FIXES NEEDED BEFORE TRAINING")
        final_recommendation = "FIX_REMAINING_ISSUES"
    
    return final_recommendation, validation_results, split_stats

# Execute re-validation after cleanup
print("🔄 Running re-validation after duplicate cleanup...")
final_recommendation, post_cleanup_validation, updated_stats = re_validate_after_cleanup(clean_split_path)

# Final status report
print(f"\n{'='*70}")
print("🎯 FINAL DATASET STATUS REPORT")
print(f"{'='*70}")

if final_recommendation == "PROCEED_WITH_TRAINING":
    print("🟢 STATUS: READY FOR TRAINING")
    print("✅ All issues have been resolved")
    print("✅ Dataset is clean, balanced, and leak-free")
    print("✅ You can now train with confidence!")
    
elif final_recommendation == "PROCEED_WITH_CAUTION":
    print("🟡 STATUS: PROCEED WITH CAUTION")
    print("⚠️  Minor issues remain but training is possible")
    print("🔍 Monitor training carefully for any anomalies")
    
else:
    print("🔴 STATUS: ADDITIONAL FIXES NEEDED")
    print("❌ Please address remaining issues before training")

print(f"\n📊 FINAL DATASET SUMMARY:")
print(f"   Total images: {sum(stats['total'] for stats in updated_stats.values()):,}")
print(f"   Training: {updated_stats['train']['total']:,}")
print(f"   Validation: {updated_stats['val']['total']:,}")
print(f"   Test: {updated_stats['test']['total']:,}")
print(f"   Classes: {len(updated_stats['train']['classes'])}")
print(f"   Duplicates removed: {duplicates_removed}")

print(f"{'='*70}")

🔄 Running re-validation after duplicate cleanup...

🔄 RE-VALIDATING AFTER DUPLICATE CLEANUP

1️⃣ RE-CHECKING FOR DUPLICATES:
----------------------------------------
   Duplicates in train: 0
   Duplicates in val: 0
   Duplicates in test: 0
   ✅ NO DUPLICATE FILES FOUND - CLEANUP SUCCESSFUL!

2️⃣ RE-CHECKING DATA LEAKAGE:
----------------------------------------
   Train files (unique): 21424
   Val files (unique): 7145
   Test files (unique): 7145
   Train-Val overlap: 0 files
   Train-Test overlap: 0 files
   Val-Test overlap: 0 files
   ✅ NO DATA LEAKAGE DETECTED!

3️⃣ UPDATED DATASET STATISTICS:
----------------------------------------
   Train: 25,537 images
   Val: 7,145 images
   Test: 7,145 images
   Total images: 39,827
   Classes: 23

4️⃣ CLASS BALANCE AFTER CLEANUP:
----------------------------------------
   Train imbalance ratio: 1.92:1
   Val imbalance ratio: 21.37:1
   Test imbalance ratio: 20.71:1
   ✅ ALL CLASSES HAVE ADEQUATE SAMPLES

5️⃣ CHECKING DATASET SIZE ADEQUAC

In [ ]:
# =============================================================================
# LIGHTWEIGHT CNN TRAINING - USING CLEAN DATA FROM FIX_MODEL_ISSUES
# =============================================================================

def create_lightweight_cnn_for_clean_data(input_shape=(224, 224, 3), num_classes=None):
    """Create lightweight CNN using the successful architecture from inspect_dataset"""
    
    model = tf.keras.models.Sequential([
        # First block
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        
        # Second block
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        
        # Third block
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        
        # Classifier
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

print("🚀 COMBINING BEST OF BOTH APPROACHES")
print("=" * 60)
print("✅ Data leakage: FIXED (from fix_model_issues.ipynb)")
print("✅ Architecture: Lightweight CNN (from inspect_dataset.ipynb)")
print("✅ Clean splits: Using clean_split_path")
print("✅ No duplicates: Already removed")

# =============================================================================
# SETUP DATA GENERATORS FOR CLEAN DATA
# =============================================================================

def setup_clean_data_generators(clean_split_path, batch_size=32, img_size=(224, 224)):
    """Setup data generators using the clean splits"""
    
    print("\n📊 Setting up data generators for clean data...")
    
    # Training generator - minimal augmentation since we already augmented
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True,
        zoom_range=0.05
    )
    
    # Val/test generators - only rescaling
    val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        os.path.join(clean_split_path, 'train'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    
    val_generator = val_test_datagen.flow_from_directory(
        os.path.join(clean_split_path, 'val'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_directory(
        os.path.join(clean_split_path, 'test'),
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    print(f"✅ Clean data generators created:")
    print(f"   Training: {train_generator.samples:,} images")
    print(f"   Validation: {val_generator.samples:,} images")
    print(f"   Test: {test_generator.samples:,} images")
    print(f"   Classes: {train_generator.num_classes}")
    
    return train_generator, val_generator, test_generator

# Setup generators using the clean splits from fix_model_issues
clean_train_gen, clean_val_gen, clean_test_gen = setup_clean_data_generators(clean_split_path)

# =============================================================================
# CREATE AND COMPILE LIGHTWEIGHT MODEL
# =============================================================================

# Create lightweight CNN
num_classes = clean_train_gen.num_classes
lightweight_model_clean = create_lightweight_cnn_for_clean_data(num_classes=num_classes)

print(f"\n🏗️ Lightweight CNN created for clean data:")
print(f"   Parameters: {lightweight_model_clean.count_params():,}")
print(f"   Classes: {num_classes}")

# Compile model
lightweight_model_clean.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"✅ Model compiled and ready for training")

# =============================================================================
# TRAINING CALLBACKS
# =============================================================================

from datetime import datetime

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-6,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f'best_lightweight_clean_again_{datetime.now().strftime("%Y%m%d_%H%M%S")}.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print(f"✅ Training callbacks configured")

# =============================================================================
# TRAIN THE MODEL
# =============================================================================

print(f"\n🚀 STARTING LIGHTWEIGHT CNN TRAINING ON CLEAN DATA")
print("=" * 60)

# Calculate steps
steps_per_epoch = clean_train_gen.samples // clean_train_gen.batch_size
validation_steps = clean_val_gen.samples // clean_val_gen.batch_size

print(f"Training setup:")
print(f"   Steps per epoch: {steps_per_epoch}")
print(f"   Validation steps: {validation_steps}")
print(f"   Max epochs: 20")
print(f"   Early stopping: patience=8")

# Start training with timing
import time
start_time = time.time()

history = lightweight_model_clean.fit(
    clean_train_gen,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_data=clean_val_gen,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

training_time = time.time() - start_time

print(f"\n⏱️ Training completed in {training_time/60:.1f} minutes")
print(f"✅ Best model saved automatically")

# =============================================================================
# EVALUATE ON TEST SET
# =============================================================================

print(f"\n📊 EVALUATING ON TEST SET")
print("=" * 40)

# Reset test generator
clean_test_gen.reset()

# Evaluate
test_loss, test_accuracy = lightweight_model_clean.evaluate(clean_test_gen, verbose=1)

print(f"\n🎯 FINAL RESULTS (NO DATA LEAKAGE):")
print(f"   Test Loss: {test_loss:.4f}")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.1f}%)")
print(f"   Training Time: {training_time/60:.1f} minutes")
print(f"   Model Parameters: {lightweight_model_clean.count_params():,}")

# Get predictions for detailed analysis
y_pred = lightweight_model_clean.predict(clean_test_gen, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = clean_test_gen.classes

# Calculate balanced accuracy
from sklearn.metrics import balanced_accuracy_score
balanced_acc = balanced_accuracy_score(y_true, y_pred_classes)

print(f"   Balanced Accuracy: {balanced_acc:.4f} ({balanced_acc*100:.1f}%)")

# =============================================================================
# RESULTS COMPARISON
# =============================================================================

print(f"\n📈 RESULTS COMPARISON:")
print("=" * 50)
print("BEFORE (inspect_dataset.ipynb):")
print("   ❌ Data leakage present")
print("   🎯 Validation accuracy: 89.45% (inflated)")
print("   ⚠️  Results not trustworthy")

print(f"\nAFTER (fix_model_issues.ipynb + lightweight CNN):")
print(f"   ✅ Data leakage: FIXED")
print(f"   ✅ Duplicates: REMOVED")
print(f"   🎯 Test accuracy: {test_accuracy*100:.1f}% (legitimate)")
print(f"   🎯 Balanced accuracy: {balanced_acc*100:.1f}% (robust)")
print(f"   ✅ Results: TRUSTWORTHY")

# Expected vs Actual
expected_drop = 10  # 10-15% drop expected after fixing data leakage
actual_accuracy = test_accuracy * 100
print(f"\n📊 EXPECTATION vs REALITY:")
print(f"   Expected after fix: 75-80% (89.45% - 10-15%)")
print(f"   Actual result: {actual_accuracy:.1f}%")

if actual_accuracy >= 75:
    print(f"   ✅ EXCELLENT - Results meet expectations!")
elif actual_accuracy >= 65:
    print(f"   🟡 GOOD - Reasonable performance")
else:
    print(f"   🔴 NEEDS IMPROVEMENT")

print(f"\n🎉 LEGITIMATE CROP DISEASE MODEL COMPLETE!")
print("=" * 60)

🚀 COMBINING BEST OF BOTH APPROACHES
✅ Data leakage: FIXED (from fix_model_issues.ipynb)
✅ Architecture: Lightweight CNN (from inspect_dataset.ipynb)
✅ Clean splits: Using clean_split_path
✅ No duplicates: Already removed

📊 Setting up data generators for clean data...
Found 25537 images belonging to 23 classes.
Found 7145 images belonging to 23 classes.
Found 7145 images belonging to 23 classes.
✅ Clean data generators created:
   Training: 25,537 images
   Validation: 7,145 images
   Test: 7,145 images
   Classes: 23


c:\Users\user\Crop Disease model\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



🏗️ Lightweight CNN created for clean data:
   Parameters: 134,103
   Classes: 23
✅ Model compiled and ready for training
✅ Training callbacks configured

🚀 STARTING LIGHTWEIGHT CNN TRAINING ON CLEAN DATA
Training setup:
   Steps per epoch: 798
   Validation steps: 223
   Max epochs: 20
   Early stopping: patience=8


c:\Users\user\Crop Disease model\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
798/798 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4644 - loss: 1.8562
Epoch 1: val_loss improved from inf to 1.72890, saving model to best_lightweight_clean_again_20250709_101912.h5


798/798 ━━━━━━━━━━━━━━━━━━━━ 4320s 5s/step - accuracy: 0.4646 - loss: 1.8556 - val_accuracy: 0.5785 - val_loss: 1.7289 - learning_rate: 0.0010
Epoch 2/20
  1/798 ━━━━━━━━━━━━━━━━━━━━ 33:49 3s/step - accuracy: 0.7188 - loss: 0.7427

c:\Users\user\Crop Disease model\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss did not improve from 1.72890
798/798 ━━━━━━━━━━━━━━━━━━━━ 85s 103ms/step - accuracy: 0.7188 - loss: 0.7427 - val_accuracy: 0.5786 - val_loss: 1.7685 - learning_rate: 0.0010
Epoch 3/20
798/798 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7462 - loss: 0.7967
Epoch 3: val_loss did not improve from 1.72890
798/798 ━━━━━━━━━━━━━━━━━━━━ 2310s 3s/step - accuracy: 0.7462 - loss: 0.7967 - val_accuracy: 0.5161 - val_loss: 2.7720 - learning_rate: 0.0010
Epoch 4/20
  1/798 ━━━━━━━━━━━━━━━━━━━━ 35:10 3s/step - accuracy: 0.7812 - loss: 0.7936
Epoch 4: val_loss did not improve from 1.72890
798/798 ━━━━━━━━━━━━━━━━━━━━ 86s 105ms/step - accuracy: 0.7812 - loss: 0.7936 - val_accuracy: 0.5929 - val_loss: 2.0445 - learning_rate: 0.0010
Epoch 5/20
798/798 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8169 - loss: 0.5626
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 5: val_loss did not improve from 1.72890
798/798 ━━━━━━━━━━━━━━━━━━━━ 2404s 3s/step - 

798/798 ━━━━━━━━━━━━━━━━━━━━ 2382s 3s/step - accuracy: 0.8748 - loss: 0.3891 - val_accuracy: 0.8295 - val_loss: 0.5981 - learning_rate: 5.0000e-04
Epoch 8/20
  1/798 ━━━━━━━━━━━━━━━━━━━━ 31:56 2s/step - accuracy: 0.8750 - loss: 0.2839
Epoch 8: val_loss did not improve from 0.59813
798/798 ━━━━━━━━━━━━━━━━━━━━ 96s 117ms/step - accuracy: 0.8750 - loss: 0.2839 - val_accuracy: 0.8205 - val_loss: 0.6422 - learning_rate: 5.0000e-04
Epoch 9/20
798/798 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8953 - loss: 0.3218
Epoch 9: val_loss did not improve from 0.59813
798/798 ━━━━━━━━━━━━━━━━━━━━ 2523s 3s/step - accuracy: 0.8953 - loss: 0.3218 - val_accuracy: 0.7241 - val_loss: 0.9862 - learning_rate: 5.0000e-04
Epoch 10/20
  1/798 ━━━━━━━━━━━━━━━━━━━━ 48:45 4s/step - accuracy: 0.8750 - loss: 0.2250
Epoch 10: val_loss did not improve from 0.59813
798/798 ━━━━━━━━━━━━━━━━━━━━ 121s 148ms/step - accuracy: 0.8750 - loss: 0.2250 - val_accuracy: 0.7309 - val_loss: 0.9560 - learning_rate: 5.0000e-04
Epoch